# Simulation tools

There are several tools available for simulating realizations of probabilistic objects &mdash; outcomes from a probability space, events, values of random variables or random vectors, stochastic processes &mdash; and analyzing the results.  In general, the methods below can be chained, one after the other, e.g. `P.sim(1000).apply(sum).tabulate()`.

<a id='contents'></a>

1. [**Simulate:**](#sim) `.sim()` simulates outcomes of a probability model, events, or values of random variables or processes
1. [**Apply:**](#apply) `.apply()` applies a function to each outcome or value
1. [**Tabulate:**](#tabulate) `.tabulate()` creates a table summary of simulated outcomes of a probability model or simulated values of random variables
1. [**Count:**](#count) `.count()` and its relatives count the values which satsify some criteria.
1. [**Filter:**](#filter) `.filter()` and its relatives create *subsets* of the values which satsify some criteria.
1. [**Get:**](#get) `.get()` returns the results of a particular realization of the simulation.
1. [**Summary of commands:**](#recap)  Examples using `tabulate`, `filter`, and `count` to manipulate simulation results.

Be sure to import Symbulate using the following commands.
<a id='simulation'></a>

In [1]:
from symbulate import *
%matplotlib inline

<a id='sim'></a>

### Simulate

The `.draw()` extension simulates one realization of the simulation.  Many realizations can be simulated using `.sim()`; the single argument is the number of repetitions of the simulation to run. (Note that [`draw`](probspace.html#draw) is most useful when defining probability spaces, while `sim` is most useful when actually running simulations.)

*Example.*  Simulate 100 repetitions of rolling two fair six-sided dice; each repetition involves a pair of values.

In [2]:
die = list(range(1, 6+1)) # this is just a list of the number 1 through 6
roll = BoxModel(die, size = 2)
roll.sim(100)

0,"(1, 3)"
1,"(3, 1)"
2,"(3, 6)"
3,"(4, 2)"
4,"(3, 1)"
5,"(5, 3)"
6,"(5, 6)"
7,"(6, 3)"
8,"(5, 6)"
...,...
99,"(2, 5)"


**Caution:** Note that every time `.sim()` is called new realizations are simulated.  Store simulated realizations as variables in order to perform multiple operations in different lines of code on the same set of simulated values.

*Example.*  Ten percent of all e-mail is spam. Thirty percent of spam e-mails contain the word "money", while 2% of non-spam e-mails contain the word "money". Simulate the email status (spam or not) and wording (money or not) for 1000 emails.

In [3]:
def spam_sim():
    email_type = BoxModel(["spam", "not spam"], probs=[.1, .9]).draw()
    if email_type == "spam":
        has_money = BoxModel(["money", "no money"], probs=[.3, .7]).draw()
    else:
        has_money = BoxModel(["money", "no money"], probs=[.02, .98]).draw()
    return email_type, has_money

P = ProbabilitySpace(spam_sim)
sims = P.sim(1000)
sims

0,"('not spam', 'no money')"
1,"('not spam', 'no money')"
2,"('not spam', 'no money')"
3,"('not spam', 'no money')"
4,"('not spam', 'no money')"
5,"('not spam', 'no money')"
6,"('not spam', 'no money')"
7,"('not spam', 'no money')"
8,"('not spam', 'no money')"
...,...
999,"('not spam', 'no money')"


<a id='apply'></a>

### Apply

Use `.apply()` to apply a function on an outcome-by-outcome basis to each realization of a simulation.

*Example.* Roll two fair six-sided dice and compute their sum.

In [4]:
die = list(range(1, 6+1)) # this is just a list of the number 1 through 6
roll = BoxModel(die, size = 2)
roll.sim(1000).apply(sum)

0,11
1,6
2,10
3,5
4,5
5,4
6,4
7,3
8,8
...,...
999,5


User defined functions can also be applied.  Standard Python commands can be used to define functions.

*Example.* Ten cards labeled 1, 2, $\ldots$, 10 are shuffled and dealt one at a time.  Find the the probability that the number on the card matches its position in the deal for at least one of the cards.  (For example, a match occurs if card 3 is the third card dealt.)

First we define a probability space whose outcomes correspond to ordered deals of the cards.  Each realization of the simulation is a permutation of the numbers 0, 1, $\ldots$ 9 (using Python indexing).  Simulate 10000 realizations and store as `sims`.

In [5]:
n = 10
labels = list(range(n)) # remember, Python starts the index at 0, so the cards are labebeled 0, ..., 9
P = BoxModel(labels, size = n, replace = False)
sims = P.sim(10000)
sims

0,"(5, 9, 4, 0, 3, ..., 6)"
1,"(3, 7, 9, 1, 2, ..., 0)"
2,"(9, 8, 5, 7, 3, ..., 1)"
3,"(7, 0, 2, 6, 3, ..., 8)"
4,"(8, 6, 0, 4, 9, ..., 1)"
5,"(9, 7, 3, 4, 1, ..., 2)"
6,"(3, 6, 1, 4, 9, ..., 2)"
7,"(8, 6, 5, 9, 7, ..., 1)"
8,"(1, 8, 7, 5, 2, ..., 6)"
...,...
9999,"(2, 6, 7, 5, 3, ..., 8)"


Next we define a function `is_match` which takes as an input a permutation of 0, $\ldots$, 9 and checks if there is at least once match.  (Defining custom functions like this is the main way that Python code is used with Symbulate.)

In [6]:
def is_match(x):
    for i in range(n):
        if x[i] == labels[i]:
            return 'At least one match'
    return 'No match'
    
is_match((1, 2, 0, 3, 5, 6, 7, 8, 9, 4)) # for the outcome, card 3 matches position 3

'At least one match'

Now we apply the `is_match` function to the simulated realizations, stored in `sims`.

In [7]:
sims.apply(is_match)

0,No match
1,At least one match
2,At least one match
3,At least one match
4,No match
5,No match
6,No match
7,No match
8,No match
...,...
9999,No match


Finally, there are various ways to estimate the probability of at least one match, which will be described below.  For example, the "match/no match" results could be tabulated to determine the relative frequence of a match.

In [8]:
sims.apply(is_match).tabulate(normalize = True)

At least one match,0.6253
No match,0.3747
Total,1.0


<a id='tabulate'></a>

### Tabulate

The results of a simulation can be quickly tabulated using `.tabulate()`.  Tabulate counts the number of times each particular outcome occurs among the simulated realizations.  Use `.tabulate(normalize=True)` to find the proportion (relative frequency) of times each outcome occurs.

**Example.** Roll two fair four-sided.  Each realization is an ordered pair of rolls.  There are 16 possible ordered pairs - (1, 1), (1, 2), ..., (4, 3), (4, 4) - all equally likely.

In [9]:
die = list(range(1, 4+1, 1)) # this is just a list of the numbers 1 through 4
roll = BoxModel(die, size=2)
rolls = roll.sim(16000)
rolls.tabulate()

"(1, 1)",1061
"(1, 2)",990
"(1, 3)",980
"(1, 4)",1044
"(2, 1)",989
"(2, 2)",984
"(2, 3)",971
"(2, 4)",1027
"(3, 1)",954
"(3, 2)",1025
"(3, 3)",972


Now sum the dice, and approximate the probability distribution of the sum using tabulate with the normalize option.

In [10]:
rolls.apply(sum).tabulate(normalize=True)

2,0.0663125
3,0.1236875
4,0.182375
5,0.2515625
6,0.18675
7,0.1258125
8,0.0635
Total,1.0


Individual entries of the table can be referenced using `.tabulate()[label]` where label represents the value of interest.

In [11]:
rolls.tabulate()[(2,4)]

1027

In [12]:
roll_sum = rolls.apply(sum).tabulate(normalize=True)
roll_sum[6] + roll_sum[7] + roll_sum[8]

0.37606249999999997

By default, tabulate only tabulates those outcomes which are among the simulated values, rather than all possible outcomes.  An argument can be passed to `.tabulate()` to tabulate all outcomes in a given list.

In [13]:
die = list(range(1, 4+1, 1)) # this is just a list of the number 1 through 4
rolls = BoxModel(die).sim(2)
rolls.tabulate(die)

1,1
2,0
3,1
4,0
Total,2


In [14]:
# Compare with
rolls.tabulate()

1,1
3,1
Total,2


By default, the outcomes in the table produced by `.tabulate()` are in alphanumeric order.  A list can be passed to `.tabulate()` to achieve a specified order.

In [15]:
BoxModel(['a', 'b', 1, 2, 3]).sim(10).tabulate([3, 'a', 2, 'b', 1])

3,1
a,6
2,1
b,0
1,2
Total,10


<a id='count'></a>

### Count

You can count the number of simulated realizations equal to a particular value using `count_eq()`.

In [16]:
BoxModel(['H','T']).sim(10000).count_eq('H')

5017

In [17]:
die = list(range(1, 4+1, 1)) # this is just a list of the number 1 through 4
roll = BoxModel(die, size = 2)
rolls = roll.sim(16000)
rolls.count_eq((2,4))

1012

In addition to `.count_eq()`, the following count functions can be used when the values are numerical.

* `.count_neq()` counts the values *not equal to* the argument
* `.count_lt()` counts the values *less than* the argument
* `.count_leq()` counts the values *less than or equal to* the argument
* `.count_gt()` counts the values *greater than* the argument
* `.count_geq()` counts the values *greater than or equal to* the argument

In [18]:
rolls.apply(sum).count_geq(6) / 16000

0.3731875

You can also count the number of outcomes which satisfy some criteria specified by a user defined function. Define a function that returns `True` for the outcomes you want to count, and pass the function into `.count()`. For example, the following code is equivalent to using `.count_geq(6)`.

In [19]:
def greater_than_or_equal_to_6(x):
    return x >= 6

rolls.apply(sum).count(greater_than_or_equal_to_6) / 16000

0.3731875

Custom functions can also be used to count based on multiple criteria.  For example, the following counts the pairs of rolls in which the first roll is equal to 2 and the second roll is at most 3.

In [20]:
def custom_count(x): # x represents a pair of values (x[0], x[1])
    if (x[0] == 2) & (x[1] <= 3):
        return True

rolls.count(custom_count)

3112

Counting can also be accomplished by creating logical (True = 1, False = 0) values according to whether an outcome satisfies some criteria and then summing.

In [21]:
rollsums = rolls.apply(sum)
sum(rollsums >= 6) / 16000

0.3731875

Since a mean (average) is the sum of values divided by the number of values, changing sum to mean in the above method returns the relative frequency directly (without having to divide by the number of values).

In [22]:
mean(rollsums >= 6)

0.3731875

<a id='filter'></a>

### Filter

You can get the subset of simulated realizations equal to a particular value using `.filter_eq()`.

In [23]:
Heads = BoxModel(['H','T']).sim(10000).filter_eq('H')
Heads

0,H
1,H
2,H
3,H
4,H
5,H
6,H
7,H
8,H
...,...
4915,H


Using `len` (length) with the filter functions is one way to count the simulated occurrences of outcomes which satisfy some criteria.

In [24]:
len(Heads)

4916

In addition to `.filter_eq()`, the following filter functions can be used when the values are numerical.

* `.filter_neq()` subsets the values *not equal to* the argument
* `.filter_lt()` subsets the values *less than* the argument
* `.filter_leq()` subsets the values *less than or equal to* the argument
* `.filter_gt()` subsets the values *greater than* the argument
* `.filter_geq()` subsets the values *greater than or equal to* the argument

In [25]:
die = list(range(1, 1+4)) # this is just a list of the number 1 through 4
sims = BoxModel(die, size=2).sim(1000).apply(sum)
len(sims.filter_geq(6)) / 1000

0.357

You can also define your own custom filter function. Define a function that returns `True` for the outcomes you want to keep, and pass the function into `.filter()`. For example, the following code is equivalent to using `.filter_geq(6)`.

In [26]:
def greater_than_or_equal_to_6(x):
    return x >= 6

len(sims.filter(greater_than_or_equal_to_6)) / 1000

0.357

<a id='get'></a>

### Get

The outcome of a particular repetition of the simulation can be accessed with `.get()`.  Recall that Python starts an index at 0, so `.get(0)` returns the first simulated value, `.get(1)` the second, etc.

In [27]:
die = list(range(1, 4+1)) # this is just a list of the number 1 through 4
roll = BoxModel(die, size = 2)
sims = roll.sim(4)
sims

0,"(4, 4)"
1,"(4, 1)"
2,"(1, 4)"
3,"(1, 1)"


In [28]:
sims.get(0)

(4, 4)

In [29]:
sims.get(2)

(1, 4)

<a id='recap'></a>

### Summary of commands

The tabulate, count, and filter functions in Symbulate allow for several ways of manipulating the outcomes of a simulation.  As an example, here is a recap of some of the ways Symbulate can be used to estimate the probability that the sum of two fair four-sided dice is at least 6.

In [30]:
die = list(range(1, 4+1, 1)) # this is just a list of the number 1 through 4
roll = BoxModel(die, size = 2)
rolls = roll.sim(10000)
rollsums = rolls.apply(sum)

In [31]:
rollsums.count_geq(6) / 10000

0.3749

In [32]:
(rollsums.tabulate()[6] + rollsums.tabulate()[7] + rollsums.tabulate()[8]) / 10000

0.3749

In [33]:
(rollsums.tabulate(normalize=True)[6] + rollsums.tabulate(normalize=True)[7] + rollsums.tabulate(normalize=True)[8])

0.3749

In [34]:
len(rollsums.filter_geq(6)) / 10000

0.3749

In [35]:
sum(rollsums >= 6) / 10000

0.3749

In [36]:
mean(rollsums >= 6)

0.3749

We will also see that [Random variables (RV)](rv.html) and events play an extremely important roll in defining and simulating probabilistics objects.  The following code (1) defines a probability space corresponding to two rolls of a fair four-sided die, (2) define a RV X equal to the sum of the rolls, (3) simulates realizations of X and uses a count function.

In [37]:
die = list(range(1, 4+1, 1)) # this is just a list of the number 1 through 4
roll = BoxModel(die, size = 2)
X = RV(roll, sum)
X.sim(10000).count_geq(6) / 10000

0.3693

We can also simulate and summarize realizations of the event (X >= 6).

In [38]:
(X >= 6).sim(10000).tabulate(normalize=True)

False,0.624
True,0.376
Total,1.0
